In [34]:
from langchain.document_loaders import TextLoader


In [35]:
loader = TextLoader('test1.txt')

In [36]:
# # 
# from langchain_community.document_loaders import PDFPlumberLoader

# import glob

# # 配置信息
# PDF_FILES_PATH = 'testpdf.pdf'
# EMBEDDING_MODEL = "nomic-embed-text"
# CHROMA_DB_PATH = 'chroma_db'
# CHUNK_SIZE = 1000
# CHUNK_OVERLAP = 0


# pdf_files = glob.glob(PDF_FILES_PATH)

# for file in pdf_files:
#     loader = PDFPlumberLoader(file)

In [37]:
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain_community.embeddings import OllamaEmbeddings

documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=50)
texts = text_splitter.split_documents(documents)
# embeddings = OpenAIEmbeddings()
EMBEDDING_MODEL = "nomic-embed-text"
local_embeddings = OllamaEmbeddings(model=EMBEDDING_MODEL)
db = FAISS.from_documents(texts, local_embeddings,normalize_L2=True)

In [38]:
retriever = db.as_retriever(search_type="similarity_score_threshold", search_kwargs={"score_threshold": .1})

In [39]:
docs = retriever.get_relevant_documents("允思拓公司的市场定位是什么？")

In [40]:
query="允思拓公司的市场定位是什么？"
db.similarity_search_with_relevance_scores(query)

[(Document(id='092a7702-0011-49eb-adcf-01998058b5d2', metadata={'source': 'test1.txt'}, page_content='允思拓生物科技售前AI工程师培训资料\n第一部分：公司主营业务介绍\n1.1 公司概况\n允思拓（天津）生物科技有限公司成立于2023 年 4 月，注册资本 200 万元，总部位于天津经济技术开发区滨海 - 中关村科技园。公司聚焦微生物、基因组及转录组技术领域，依托生物信息学与 AI 技术整合，作为一家聚焦生物科技领域的创新型企业，致力于为高校、科研院所、医疗机构及农业企业提供专业化服务。\n1.2 主要产品与服务\n•\t微生物：\n1、病原微生物监测预警；\n技术应用：开发新冠病毒监测预警技术、基于高通量测序的病原细菌溯源算法，实现快速病原体鉴定与传播链追踪；结合基因组、转录组数据，构建病原微生物进化树，辅助疫情防控与溯源决策。\n2、草图\n3、完成图\n4、扩增子\n5、宏基因组\n•\t基因组：\n1、全基因组survey\n2、基因组组装、注释\n3、动植物基因组T2T分析\n4、图形泛基因组分析\n5、比较基因组分析\n6、Hic辅助染色体挂载分析\n7、全基因组甲基化分析\n8、全基因组Gwas分析\n•\t转录组：\n1、原核/真核转录组分析\n2、Pacbio全长转录组分析\n3、宏转录组分析\n4、smallRNA/lncRNA分析\n1.3 技术优势\n•\t核心技术：\n1、核心团队建立的基于高通量测序的病原微生物检测溯源的算法和流程，连续三年在“联合国秘书长调查机制”能力评测中表现优异，获得相关部委表彰。疫情期间新冠监测报告多次被中央办公厅采用，并获中央领导批示。\n2、公司核心团队构建超大复杂基因组组装算法和策略。与中国科学院海洋研究所、中国林业科学研究院、河南省农科院等单位合作完成了包括对虾、海参、扇贝、藏野驴，芝麻和杨树等多种复杂大基因组的测序工作。\n3、公司核心团队平均具有12.5年行业从业经验。长期从事基因组学及生物信息学研究，专注于群体基因组进化和变异监测领域。累计开发了包括高复杂度海洋生物组装技术、A2基因型奶牛筛选技术、病原微生物检测溯源等算法和流程。'),
  0.42429019158416725),
 (Docum

In [41]:
from langchain_ollama import ChatOllama

model_name="deepseek-r1:8b"
llm = ChatOllama(model=model_name,
                            base_url="http://10.168.6.88:11434",temperature = 0,mirostat_tau=2.0,top_k=10,top_p=0.5
                            )

In [42]:
# Helper function for printing docs


def pretty_print_docs(docs):
    print(
        f"\n{'-' * 100}\n".join(
            [f"Document {i+1}:\n\n" + d.page_content for i, d in enumerate(docs)]
        )
    )

In [43]:
# 另一种更简单但更强大的过滤器
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainFilter

_filter = LLMChainFilter.from_llm(llm)
compression_retriever = ContextualCompressionRetriever(
    base_compressor=_filter, base_retriever=retriever
)

compressed_docs = compression_retriever.invoke(
    "我要做细菌框架图（1G）,大概要多少钱？"
)
pretty_print_docs(compressed_docs)

Document 1:

第二部分：价格体系
2.1 产品定价
     
样本量	<=10	<=50	<=100	数据量+1G	周期
meta（6G）	400	380	350	50	30d-40d
真核转录组（6G）	400	380	350	50	30d-40d
16S（5wtags）	100	90	80		30d-40d
原核重测序（1G）	250	220	200	30	30d-40d
真核重测序（3G）	300	280	260	30	30d-40d
细菌完成图(ONT)	1500	1300	1100		30d-40d
细菌完成图(Pacbio)	2800	2500	2200		30d-40d
细菌框架图（1G）	280	260	230		30d-40d
真菌框架图（3G）	500	480	450	150	30d-40d
真菌精细图（ONT）（3G）	3000	2800	2500	1000	30d-40d
真菌精细图（PB）（3G）	5000	4800	4500	1500	30d-40d
三代meta（ONT）（10G）	3000	2900	2800		30d-40d
二代宏转录组 (非病毒6Gb)	1600	1400	1200		30d-40d
二代宏转录组 (病毒6Gb)	1800	1600	1400		30d-40d
原核转录组（3G）	700	650	600		30d-40d


In [44]:
query="我要做细菌框架图（1G）,大概要多少钱？"
db.similarity_search_with_relevance_scores(query)

[(Document(id='90973b13-770b-41e9-8b32-a09ca244c73a', metadata={'source': 'test1.txt'}, page_content='•\t研发团队：团队成员熟练掌握企业管理所需的各类专业知识，包括涉及各基因组技术的产品开发、生产扩大、商业扩张和全球客户参与业务。主要管理人员在相关领域深耕10年以上，积累了丰富的分析和开发经验。我们坚持紧跟科研趋势，不断创新，为科研工作者提供最具影响力的科学成果和技术洞察。\n•\t知识产权：NA\n1.4 市场定位\n•\t目标客户：高校、科研院所、医疗机构及农业企业\n•\t竞争优势：\n1核心成员平均拥有10多年的行业经验，专注微生物、动植物基因组测序分析及算法开发，多大10余项核心技术，包括新冠病毒监测预警、病原细菌溯源算法等；\n2价格相对优势；首先我们人工成本低，人效高，比如大规模这种公司，管理层和人力等会占实际产生利润的人员2：3，也就是说他必须卖高价才可以维持公司正常运转\n3 我们售后服务相对优势；我们团队都是专业深耕这个领域很多年，有丰富的项目经验，比如大规模公司他们好多都是新培养的新人去做，而且项目巨多，没有多余时间去处理售后项目\n4 项目周期相对优势；大公司会有运营对接客户，这种会延迟项目问题的解决及准确的传达问题点，我们采用信息直接对接客户的方式，能快速解决客户的痛点及问题'),
  0.39923207782995473),
 (Document(id='f6ea58f6-5149-4a2a-9911-678b327c2832', metadata={'source': 'test1.txt'}, page_content='第二部分：价格体系\n2.1 产品定价\n     \n样本量\t<=10\t<=50\t<=100\t数据量+1G\t周期\nmeta（6G）\t400\t380\t350\t50\t30d-40d\n真核转录组（6G）\t400\t380\t350\t50\t30d-40d\n16S（5wtags）\t100\t90\t80\t\t30d-40d\n原核重测序（1G）\t250\t220\t200\t30\t30d-40d\n真核重测序（3G）\t300\t280\t260\t30\t30d-40d\n细菌完